- 모델에서 중요한 매개변수를 찾는건 필수적인일. 가장 많이 사용하는 방법은 그리드서치 
- 그리드 서치 : 관심 있는 매개변수를 대상으로 가능한 모든 조합을 시도해봄. 
- svm 으로 예시를 들건데 ,  svm은 커널의 폭에 해당하는 gamma와 규제 매개변수 C가 중요함.


In [1]:
# 매개변수 C와 gamma에 0.001, 0.01, 0.1, 1. 10. 100 값을 적용해볼것임 
# C와 gamma의 설정값이 각각 6개이니 조합은 36개가 나옴. 

# 간단한 그리드 서치 구현 
import pandas as pd 
import numpy as np 

df = pd.read_csv('./cold_high_preprocessed.csv',encoding='cp949',index_col=0)
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
y = df.재난일어난날
x = df.drop('재난일어난날',axis=1)
x_train, x_test,y_train, y_test = train_test_split(x,y,random_state=0)
print("훈련 세트의 크기: {} 테스트 세트의 크기: {}".format(x_train.shape[0], x_test.shape[0]))

훈련 세트의 크기: 8755 테스트 세트의 크기: 2919


In [3]:
# best_score = 0 

# for gamma in [0.001,0.01,0.1,1,10,100]:
#     for C in [0.001,0.01,0.1,1,10,100]:
#         svm = SVC(gamma=gamma, C=C)
#         svm.fit(x_train,y_train)
#         score =svm.score(x_test,y_test)
#         if score > best_score:
#             best_score = score
#             best_parameters = {'C': C, 'gamma': gamma}
# print("최고점수:{:.2f}".format(best_score))
# print("최적 매개변수:",best_parameters)

- 앞의 결과를 보면 모델정확도는 87퍼임 . 
- 하지만 이결과는 잘못된것일수있음
- 여러가지 매개변수 값으로 시도해보고 테스트세트정확도가 가장 높은 조합을 선택함. 
- 하지만 이 정확도는 새로운 데이터까지 이어지지 않을 수 있음 . 
- 평가를 위해선 독립된 데이터 셋이 필요함.
- 그러므로 데이터를 다시 나눠서 세개의 세트로 만들어 이 문제를 해결 할 수 있음. 
#### 훈련세트로는 모델을 만들고 , 검증 세트로는 모델의 매개변수를 선택하고 , 테스트 세트로는 선택된 매개 변수의 성능을 평가함.
- 검증세트로 최적의 매개변수를 선택한 후 , 그 매개변수에서 훈련세트와 검증세트데이터를 모두 이용해 모델을 다시만듬. 
- => 이렇게 하는 이유는 모델을 만들때 가능한 많은 데이터를 이용하기 위해서임.

In [4]:
from sklearn.svm import SVC 
# 데이터를 훈련+검증 세트 그리고 테스트 세트로 분할 
x_trainval, x_test, y_trainval, y_test = train_test_split(x,y,random_state=0)
# 훈련_검증 세트를 훈련 세트와 검증 세트로 문할 
x_train, x_valid,y_train, y_valid = train_test_split(x_trainval, y_trainval, random_state=0)

print("훈련세트의 크기: {} 검증세트의 크기 :{} 테스트 세트의 크기 : {}\n".format(x_train.shape[0], x_valid.shape[0],x_test.shape[0]))

best_score = 0 

for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        # 매개변수의 각 조합에 대해 SVC를 훈련시킴. 
        svm = SVC(gamma=gamma, C=C)
        svm.fit(x_train,y_train)
        # 검증 세트로 SVC를 평가함 
        score = svm.score(x_valid, y_valid)
        # 점수가 더 높으면 매개변수와 함께 기록함. 
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}
            
        #훈련세트와 검증 세트를 합쳐 모델을 다시 만든 후 
        # 테스트 세트를 사용해 평가함 
svm = SVC(**best_parameters)
svm.fit(x_trainval, y_trainval)
test_score = svm.score(x_test, y_test)
print("검증 세트에서 최고 점수 : {:.2f}".format(best_score))
print("최적 매개변수:",best_parameters)
print("최적 매개변수에서 테스트 세트 점수: {:.2f}".format(test_score))

훈련세트의 크기: 6566 검증세트의 크기 :2189 테스트 세트의 크기 : 2919

검증 세트에서 최고 점수 : 0.89
최적 매개변수: {'C': 0.001, 'gamma': 0.001}
최적 매개변수에서 테스트 세트 점수: 0.87
